# Teil 1: Laden und Visualisierung der Daten
Der erste Praxisteil beschäftigt sich mit den Daten für das spätere Training. Hier lernen Sie, wie Sie sich mit den Bildern vertraut machen, sie laden, anzeigen und normalisieren. Dabei haben Sie die Gelegenheit, die von Pytorch bereitgestellten Werkzeuge dataset, transformations und dataloader zu kennenzulernen.

In [ ]:
"""
Dieser Code kopiert und importiert notwendige Dateien in die virtuelle Maschine von Colab.
"""
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/Criscraft/workshop_ki_hautkrebserkennung.git'
    os.chdir('workshop_ki_hautkrebserkennung')

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import sys
import torch
import matplotlib.pyplot as plt
import utils
import numpy as np
%matplotlib notebook

# TODO: Geben Sie das Verzeichnis an, in dem sich der Datensatz befindet
DATA = 'data/test/'

## 1. Laden der Daten mittels der ImageFolder Klasse
Erstellen Sie eine Instanz der Klasse ```ImageFolder``` Geben Sie dabei das Verzeichnis ```DATA``` als Speicherort der Daten an.

Die ImageFolder Klasse bietet die Möglichkeit, bequem einzelne Datenpunkte zu laden. Außerdem wendet sie eine von Ihnen gewählte Transformation auf jedes Bild an. Damit lässt sich ein Datenpunkt z.B. normalisieren, andersweitig vorverarbeiten oder augmentieren. Wir wenden die Transformation ``` transforms.ToTensor() ``` an, um die Bilder in Pytorch Tensoren umzuwandeln, mit denen die GPU rechnen kann.

In [ ]:
# Laden der Daten
# Erstellen Sie eine Transformation zur Umwandlung in einen Pytorch Tensor
transform_to_tensor = transforms.ToTensor()
# Erstellen Sie zwei datasets trainset und testset
dataset = ImageFolder(DATA, transform=transform_to_tensor)
# Bezeichnungen der Klassen
label_names = ('mel', 'nv')

## Aufgabe

- Wie viele Bilder sind in dem Trainings- und Testdatensatz enthalten?
- Geben Sie die Größe eines Bildes aus (shape).
- Geben Sie einige Bildlabel aus. Wie sind die Label kodiert?
- Wie viele Klassen enthält jeder Datensatz? Wie viele Bilder pro Klasse?

In [ ]:
# TODO: Machen Sie sich mit dem Datensatz vertraut.


## 2. Plotten der Bilder
Nun wollen wir uns einige der Trainingsbilder anzeigen lassen, um einen Eindruck über die Daten zu gewinnen. Nutzen Sie dafür die Bibliothek matplotlib. Wenn Sie fertig sind, sollte Ihr Code folgendes leisten:

1. Einen Plot mit 4x4 Kacheln erstellen.
2. Wählen Sie in der for-Schleife ein Bild aus dem Datensatz aus und transformieren Sie es zu einem PIL Bild. Für die Transformation erstellen Sie zuerst ein Transformationsobjekt mit ```transforms.ToPILImage()``` und verwenden Sie dieses Objekt anschließend wie eine Funktion, die Ihnen einen transformierten Datenpunkt zurückgibt. 
3. In der Schleife legen Sie den Titel der Kachel mit dem Label fest.
4. Stellen Sie sicher, dass Matplotlib kein Koordinatengitter einfügt und plotten Sie das PIL Bild mit der Methode.

Folgende Funktionen könnten für Sie interessant sein: plt.subplots, transforms.ToPILImage(), ax.set_title, plt.axis('off'), ax.imshow

In [ ]:
# Erstellen Sie eine Transformation, die einen Bildtensor in einem PIL Bild überführt
transform_tensor_to_pil = transforms.ToPILImage()

fig, _ = plt.subplots(4, 4, figsize=(9, 9))

for i in range(16):
    # Auswahl der zu bearbeitenden Kachel
    ax = plt.subplot(4, 4, i+1)
    # Plotten Sie das i-te Bild im dataset mit dem Label als Titel
    ax.set_title("label " + str(dataset[i][1]) + " (" + label_names[dataset[i][1]] + ")" )
    print_image = transform_tensor_to_pil(dataset[i][0])
    ax.imshow(print_image)
    plt.axis('off')

## Aufgabe

- Wie gut eignen sich die Bilder für automatisierte Bilderkennung?
- Benötigt man lokale oder globale Merkmale für die Klassifikation?
- Wie sollte man die Bilder am besten vorverarbeiten?

## 3. Die Dataloader Klasse

Nun wollen wir die Klasse ```torch.utils.data.DataLoader``` kennenlernen und das Plotten damit wiederholen. Mit einem Dataloader lässt sich das Laden der Bilder und das Verpacken zu Minibatches automatisieren. Minibatches sind Pakete aus mehreren Trainingsbildern, die wir auf der Grafikkarte verarbeiten wollen. Das ist schneller, als die Bilder einzelnd zup Training zu verwenden. Somit kann man die Bilder mithilfe der CPU laden und vorverarbeiten, während das eigentliche Training zeitgleich auf der GPU durchgeführt wird. Um den Umgang mit dem Dataloader zu üben, erstellen wir einen Dataloader, extrahieren den ersten Minibatch und stellen ihn wieder in einem 4x4 Layout dar. Ihr Code sollte

1. Einen Dataloader mit dem Trainingsdatensatz und einer Batchgröße von 16 erstellen
2. Den ersten Batch extrahieren (Tipp: ```batch = next(iter(loader))```.
3. Die Bilder und die Labels trennen (Tipp: was befindet sich in ```batch[0]``` und ```batch[1]```?)
4. Wie zuvor in einer for-Schleife die Bilder anzeigen. Sie müssen vor dem Anzeigen eines Bildes, den Bildtensor mit einer Transformation ```transforms.ToPILImage()``` in ein anzeigbares Bild umwandeln.


In [ ]:
# Erstellen Sie einen Dataloader mit Batchsize 16
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
# Erstellen Sie eine Transformation, die einen Bildtensor in einem PIL Bild überführt
transform_tensor_to_pil = transforms.ToPILImage()

fig, _ = plt.subplots(4, 4, figsize=(9, 9))

# Extrahieren Sie den ersten Minibatch aus dem Dataloader
batch = next(iter(loader))
# Trennen Sie die Bilddaten von ihren Labels
images = batch[0]
targets = batch[1]

# Plotten Sie die 16 Bilder
for i in range(16):
    # Auswahl der zu bearbeitenden Kachel
    ax = plt.subplot(4, 4, i+1)
    ax.set_title("label " + str(int(targets[i])) + " (" + label_names[targets[i]] + ")" )
    print_image = transform_tensor_to_pil(images[i])
    ax.imshow(print_image)
    plt.axis('off')

## Aufgabe

- Wie viele Batches besitzt der Dataloader?
- Wie ist ein Batch aufgebaut und wie groß ist ein Batch?
- Warum werden nicht dieselben Bilder angezeigt wie bei der Zelle davor?

In [ ]:
# TODO: Machen Sie sich mit dem Dataloader vertraut



## 4. Augmentieren Sie Ihre Bilder

Wenn Sie nicht genügend gelabelte Daten zur Verfügung haben, kann es sein, dass Ihr neuronales Netz die Trainingsbilder schlicht auswendig lernt. Um das zu verhindern, können wir die Menge an Bildern künstlich erhöhen. Hier werden wir die Bilder augmentieren. Hierfür werden wir eine Transformation erstellen, die weißes Rauschen über die Bilder legt. Anschließend lassen wir uns die augmentierten Bilder anzeigen.

1. Erstellen Sie die Transformationen ```utils.AddGaussianNoise(blend_alpha_range=(0., 0.4))``` und ```transforms.ToTensor()``` und fassen Sie sie mittels ```transforms.Compose``` zu einer Transformation zusammen.
2. Erstellen Sie erneut ein dataset mit dieser Transformation.
3. Erstellen Sie wie in der oberen Zelle einen dataloader und plotten Sie 16 augmentierte Bilder.

In [ ]:
# Erstellen Sie eine Transformation, die erst Weißes Rauschen dem Bild hinzufügt und anschließend das Bild in einen Tensor überführt.
transformations = transforms.Compose([
    transforms.Resize((128,128)),
    utils.AddGaussianNoise(blend_alpha_range=(0.6, 0.6)),
    transforms.ToTensor(),
    ])
# Erstellen Sie ein neues dataset mit der neuen Transformation
dataset = ImageFolder(DATA, transform=transformations)

# Erstellen Sie einen dataloader und plotten Sie wie zuvor 16 Bilder.
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
transform_tensor_to_pil = transforms.ToPILImage()
fig, _ = plt.subplots(4, 4, figsize=(9, 9))

batch = next(iter(loader))
images = batch[0]
targets = batch[1]

for i in range(16):
    # Auswahl der zu bearbeitenden Kachel
    ax = plt.subplot(4, 4, i+1)
    ax.set_title("label " + str(int(targets[i])) + " (" + label_names[targets[i]] + ")" )
    print_image = transform_tensor_to_pil(images[i])
    ax.imshow(print_image)
    plt.axis('off')

## Aufgabe

- Variieren Sie die blend_alpha_range und sehen Sie den Effekt auf das Rauschen. 
- Wie stark können können Sie das Rauschen erhöhen bis Sie die Klassen nicht mehr zuverlässig erkennen?
- Was für andere Arten der Augmentierung fallen Ihnen ein?
- Wie klein können Sie die Bilder skalieren, um die Klassen mit oder ohne Verrauschen noch erkennen zu können? Das ist eine wichtige Überlegung, um später Rechenzeit einzusparen.

## 5. Normalisierung der Bilder

Sie sind nun in der Lage, Bilder zu laden, sie zu transformieren und sie mit einem Dataloader in Minibatches zu packen. Was noch fehlt, bevor Sie sich auf das Training stürzen können, ist die Normalisierung der Daten. Berechnen Sie den Wert des mittleren Pixels der Daten (das ist eine Zahl für jeweils rot, grün und blau) und dessen Standardabweichung (ebenfalls drei Zahlen). Später werden wir diese Werte verwenden, um die Bilder von ihrem Mittelwert zu bereinigen und ihre Varianz zu normalisieren. Ihr fertiger Code sollte durch das dataset iterieren und Mittelwert, sowie Standardabweichung berechnen. Hierfür steht Ihnen Welford's Onlinealgorithmus bereit. Nutzen Sie die Funktionen ```utils.update_mean_std``` und ```utils.finalize_mean_std```, um den Mittelwert und die Standardabweichung schrittweise zu erhalten. 

In [ ]:
transformations = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    ])
dataset = ImageFolder(DATA, transform=transformations)

aggregate = (0., 0., 0.)

for i in range(len(dataset)):
    #Lassen Sie sich das i-te Bild aus dem dataset geben
    #Wandeln Sie es in ein ndarray um
    #Berechnen Sie das mittlere Pixel dieses Bildes.
    image = dataset[i][0].numpy().mean(axis=(1,2))
    #Aktualisieren Sie den laufenden Mittelwert und die Standardabweichung des mittleren Pixels mit utils.update_mean_std
    aggregate = utils.update_mean_std(aggregate, image)

#Bestimmen Sie den mittleren Pixel und seine Standardabweichung.
image_means, image_stds, _ = utils.finalize_mean_std(aggregate)

print('image mean:')
print(image_means)

print('image std:')
print(image_stds)

## Aufgabe

- Was sagen image_mean und image_std aus?
- Wie würden Sie vorgehen, um ein Bild zu normalisieren?